In [2]:
import pandas as pd, numpy as np

# LSFF: basic data cleaning

IN: extraction sheet + vehicle-nutrient, country-vehicle dicts

OUT: cleaned extraction sheet. values renamed, dropped all illegal location-vehicle-val-nutrient combos

In [3]:
## load targets
import pickle
data_prep_dir = '/ihme/scratch/users/beatrixh/vivarium_data_analysis/pre_processing/lsff_project/data_prep/'

with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)

In [4]:
countries = [i for i in country_vehicle_map.keys() if type(i)==str]

In [5]:
path = data_prep_dir + 'extraction_sheet_lsff_01_13_2021.csv'
df = pd.read_csv(path, encoding = 'utf-8')

In [6]:
check_cols = ['location_id','location_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','notes']

In [8]:
# format vars
df.location_id = df.location_id.fillna(-1).astype(int)
df.location_name = df.location_name.astype(str)
df.vehicle = df.vehicle.astype(str)
df.value_description = df.value_description.astype(str)

df.value_mean = df.value_mean.mask(df.value_mean.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_025_percentile = df.value_025_percentile.mask(df.value_025_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_975_percentile = df.value_975_percentile.mask(df.value_975_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)

In [9]:
#drop all NaN-only rows
df = df.loc[~((df.value_mean.isna())&(df.value_025_percentile.isna())&(df.value_975_percentile.isna()))]

In [10]:
#locations for which need tofill in loc_ids
df[df.location_id==-1].location_name.unique()

array(['China', 'Egypt', 'South Africa', 'Nepal'], dtype=object)

In [11]:
#looks like we'll need to estimate mean here
df.loc[df.value_mean.isna(),check_cols]

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes
261,207,Ghana,rice,percent of vehicle that is industrially produced,na,NaN,11.0,23.0,"% industrially milled, only a range was given"
502,214,Nigeria,rice,percent of vehicle that is industrially produced,na,NaN,12.0,24.0,"% industrially milled, only a range was given"


## location_name

In [12]:
df.location_name.unique()

array(['India', 'Bangladesh', 'Pakistan', 'Burkina Faso', 'Nigeria',
       "Côte d'Ivoire", 'Cameroon', 'Uganda',
       'United Republic of Tanzania', 'Ethiopia', 'China',
       'Democratic Republic of the Congo', 'Egypt', 'Philippines',
       'Ghana', 'Niger', 'Yemen', 'Viet Nam', 'Sudan', 'Angola', 'Kenya',
       'Indonesia', 'Madagascar', 'Mozambique', 'Nepal', 'Myanmar',
       'South Africa', "CÃ´te d'Ivoire"], dtype=object)

In [13]:
relabel_location_name = {'India': 'India',
 'Bangladesh': 'Bangladesh',
 'Cameroon': 'Cameroon',
 'Nigeria': 'Nigeria',
 'Pakistan': 'Pakistan',
 'Burkina Faso': 'Burkina Faso',
 "CÃ´te d'Ivoire": "Côte d'Ivoire",
 "Côte d'Ivoire":"Côte d'Ivoire",
 'Uganda': 'Uganda',
 'United Republic of Tanzania': 'United Republic of Tanzania',
 'Ethiopia': 'Ethiopia',
 'China': 'China',
 'Indonesia': 'Indonesia',
 'Democratic Republic of the Congo': 'Democratic Republic of the Congo',
 'Kenya': 'Kenya',
 'Egypt': 'Egypt',
 'Niger': 'Niger',
 'Philippines': 'Philippines',
 'Ghana': 'Ghana',
 'Afghanistan': 'Afghanistan',
 'Viet Nam': 'Vietnam',
 'Yemen': 'Yemen',
 'Sudan': 'Sudan',
 'Angola': 'Angola',
 'Iraq':'Iraq',
 'Madagascar': 'Madagascar',
 'Mozambique': 'Mozambique',
 'Nepal': 'Nepal',
 'Myanmar': 'Myanmar',
 'South Africa': 'South Africa'}

In [14]:
df.location_name = df.location_name.map(relabel_location_name)

In [15]:
## exclude countries the gates foundation had us drop (politically unstable or low rates of maternal and child anemia)
excluded_countries = ['Afghanistan','Yemen','Philippines','Iraq']
df = df[~df.location_name.isin(excluded_countries)]

In [16]:
## clean country names
assert df.location_name.nunique()==25, "wrong number of countries"

In [17]:
for i in df.location_name.unique():
    if i not in countries:
        print(i)
    assert(i in countries)

## vehicle

In [18]:
##this one looks fine
df.vehicle.unique()

array(['oil', 'wheat flour', 'salt', 'rice', 'maize flour', 'bouillon'],
      dtype=object)

## nutrient

In [19]:
nutrient_map = {'vitamin a': 'vitamin a',
 'na': 'na',
 'iron': 'iron',
 'folic acid, folate, b9': 'folic acid',
 'b12': 'vitamin b12',
 'iodine': 'iodine',
 'b1, thiamine': 'vitamin b1',
 'zinc': 'zinc',
 'd, ergocalciferol-D2, cholecalciferol-D3, alfacalcidol': 'vitamin d',
 'b2, riboflavin': 'vitamin b2',
 'b3, niacin': 'vitamin b3',
 'b6, pyridoxine': 'vitamin b6'}

In [20]:
df.nutrient = df.nutrient.map(nutrient_map)

In [21]:
# fill nans
df['nutrient'] = df['nutrient'].fillna('na')

## value_description

In [22]:
df.value_description.unique()

array(['percent of population eating fortified vehicle',
       'percent of population eating industrially produced vehicle',
       'percent of population eating vehicle',
       'percent of vehicle that is industrially produced',
       'percent of vehicle that is fortified',
       'percent of marketshare of fortified products'], dtype=object)

## eliminate country-vehicle and vehicle-nutrient pairs we're uninterested in

In [23]:
country_vehicle_pairs = pd.DataFrame([(i,j) for i in countries for j in country_vehicle_map[i]], columns = ['location_name','vehicle'])

In [24]:
vehicles = list(vehicle_nutrient_map.keys())
vehicle_nutrient_pairs = pd.DataFrame([(i,j) for i in vehicles for j in vehicle_nutrient_map[i]], columns = ['vehicle','nutrient'])
vehicle_nutrient_pairs = pd.concat([vehicle_nutrient_pairs, pd.DataFrame([(i,'na') for i in vehicles], columns = ['vehicle','nutrient'])])

In [25]:
# check what we're removing
test = df.merge(vehicle_nutrient_pairs, on = ['vehicle','nutrient'], how = 'right')
[(i,j) for (i,j) in zip(df.vehicle,df.nutrient) if (i,j) not in zip(test.vehicle,test.nutrient)]

[('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('salt', 'iodine'),
 ('rice', 'vitamin a')]

In [26]:
df = df.merge(country_vehicle_pairs, on = ['location_name','vehicle'], how = 'right') #should this be inner?

In [27]:
df = df.merge(vehicle_nutrient_pairs, on = ['vehicle','nutrient'], how = 'right')

In [28]:
#drop all NaN-only rows
df = df.loc[~((df.value_mean.isna())&(df.value_025_percentile.isna())&(df.value_975_percentile.isna()))]

## clean illegal value-nutrient combos

In [29]:
nutrient_irrelevant_vds = ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle',
       'percent of vehicle that is industrially produced',
       'percent of marketshare of fortified products']

nutrient_relevant_vds = ['percent of population eating fortified vehicle',
       'percent of vehicle that is fortified',
       'percent of marketshare of fortified products']

In [30]:
df.nutrient.unique()

array(['vitamin a', 'na', 'vitamin d', 'iron', 'folic acid',
       'vitamin b12', 'zinc', 'vitamin b1'], dtype=object)

In [31]:
df.loc[(df.value_description.isin(nutrient_irrelevant_vds))&(df.nutrient!='na'),check_cols]

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes
1,161.0,Bangladesh,oil,percent of vehicle that is industrially produced,vitamin a,52.0,NaN,NaN,no uncertainty
3,161.0,Bangladesh,oil,percent of population eating industrially prod...,vitamin a,87.5,NaN,NaN,no uncertainty
17,205.0,Côte d'Ivoire,oil,percent of marketshare of fortified products,vitamin a,89.0,NaN,NaN,NaN


In [32]:
# relabel nutrients for rows for which nutrient doesn't apply
df.loc[(df.value_description.isin(nutrient_irrelevant_vds))&(df.nutrient!='na'),'nutrient'] = 'na'

In [33]:
# these need to be re extracted; for now we'll drop them
df.loc[(df.value_description.isin(nutrient_relevant_vds)) & (df.nutrient=='na'),check_cols + ['user']].sort_values(['user','location_name'])

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,notes,user
509,202.0,Cameroon,maize flour,percent of vehicle that is fortified,na,50.0,NaN,NaN,NaN,paulina
17,205.0,Côte d'Ivoire,oil,percent of marketshare of fortified products,na,89.0,NaN,NaN,NaN,paulina
197,163.0,India,wheat flour,percent of vehicle that is fortified,na,2.0,NaN,NaN,NaN,rohit
475,163.0,India,rice,percent of vehicle that is fortified,na,0.0,NaN,NaN,NaN,rohit


In [34]:
# drop rows that need nutrient filled in
df = df.loc[~((df.value_description.isin(nutrient_relevant_vds)) & (df.nutrient=='na')),]

In [37]:
# format vars
df.location_id = df.location_id.fillna(-1).astype(int)
df.location_name = df.location_name.astype(str)
df.vehicle = df.vehicle.astype(str)
df.value_description = df.value_description.astype(str)

df.value_mean = df.value_mean.mask(df.value_mean.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_025_percentile = df.value_025_percentile.mask(df.value_025_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)
df.value_975_percentile = df.value_975_percentile.mask(df.value_975_percentile.isin(['na',np.nan,-1,'unknown']), np.nan).astype(float)

In [41]:
## TODO: fix df.loc[df.location_id==-1,]

## save 

In [44]:
save_path = '/ihme/homes/beatrixh/notebooks/viv_rsc/new_lsff/outputs/lsff_data_stage0.csv'
df.to_csv(save_path, index = False)